In [1]:
import os
import os.path as op
import glob
from pathlib2 import Path
from ndmg.scripts import ndmg_dwi_pipeline
from ndmg.utils import s3_utils

In [2]:
# Download test data
data_dir = op.expanduser("~") + "/.ndmg/HNU1t"
if not op.isdir(data_dir):
    os.mkdir(data_dir)
    s3_utils.s3_get_data('ndmg-data', 'HNU1t', data_dir, force=True)

# Specify base directory and paths to input files (dwi, bvecs, bvals, and t1w required)
t1w = data_dir + '/sub-0025427/ses-1/anat/sub-0025427_ses-1_T1w.nii.gz'
bvals = data_dir + '/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.bval'
bvecs = data_dir + '/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.bvec'
dwi = data_dir + '/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.nii.gz'

###USER-SPECIFIED PARAMETERS###
# Specify NDMG run parameters
## CHOICES:
## atlases = ['desikan', 'CPAC200', 'DKT', 'HarvardOxfordcort', 'HarvardOxfordsub', 'JHU', 'Schaefer2018-200', 'Talairach', 'aal', 'brodmann', 'glasser', 'yeo-7-liberal', 'yeo-17-liberal']
## mod_types = ['det', 'prob']
## track_types = ['local', 'particle']
## mods = ['csa', 'csd']
## regs = ['native', 'native_dsn', 'mni']
## vox_size = ['1mm', '2mm']

# Select an atlas
atlas = 'desikan'
mod_type = 'det'
track_type = 'local'
mod_func = 'csd'
reg_style = 'native'
vox_size = '2mm'

# Set an output directory
outdir = '/tmp/output_{}_{}_{}_{}_{}'.format(atlas, mod_type, track_type, mod_func, reg_style)
###############################
# Download neuroparc files
atlas_dir = op.expanduser("~") + "/.ndmg/ndmg_atlases"
    
# Auto-set paths to neuroparc files
mask = str(Path.home()) + '/.ndmg/ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz'
labels = [i for i in glob.glob(str(Path.home()) + '/.ndmg/ndmg_atlases/atlases/label/Human/*2x2x2.nii.gz') if atlas in i]

# Run pipeline
ndmg_dwi_pipeline.ndmg_dwi_worker(dwi, bvals, bvecs, t1w, atlas, mask, labels, outdir, vox_size, mod_type, track_type, mod_func, reg_style, clean=False, skipeddy=False, skipreg=False)


dwi = /Users/derekpisner/.ndmg/HNU1t/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.nii.gz
bvals = /Users/derekpisner/.ndmg/HNU1t/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.bval
bvecs = /Users/derekpisner/.ndmg/HNU1t/sub-0025427/ses-1/dwi/sub-0025427_ses-1_dwi.bvec
t1w = /Users/derekpisner/.ndmg/HNU1t/sub-0025427/ses-1/anat/sub-0025427_ses-1_T1w.nii.gz
atlas = desikan
mask = /Users/derekpisner/.ndmg/ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz
labels = ['/Users/derekpisner/.ndmg/ndmg_atlases/atlases/label/Human/desikan_space-MNI152NLin6_res-2x2x2.nii.gz']
outdir = /tmp/output_desikan_det_local_csd_native
vox_size = 2mm
mod_type = det
track_type = local
mod_func = csd
reg_style = native
clean = False
skip eddy = False
Output directory: /tmp/output_desikan_det_local_csd_native
Adding directory tree...
Connectomes downsampled to given labels: /tmp/output_desikan_det_local_csd_native/dwi/roi-connectomes/desikan_space-MNI152NLin6_res-2x2x2/sub-0025427_ses-1_dwi_d

flirt -in /tmp/output_desikan_det_local_csd_native/tmp/reg_a/vent_mask_t1w.nii.gz -ref /tmp/output_desikan_det_local_csd_native/dwi/preproc/nodif_B0.nii.gz -out /tmp/output_desikan_det_local_csd_native/tmp/reg_a/t1w_vent_mask_dwi.nii.gz -init /tmp/output_desikan_det_local_csd_native/tmp/reg_m/t1wtissue2dwi_xfm.mat -interp trilinear -dof 6 -applyxfm
flirt -in /tmp/output_desikan_det_local_csd_native/anat/preproc/t1w_seg_pve_0_nores.nii.gz -ref /tmp/output_desikan_det_local_csd_native/dwi/preproc/nodif_B0.nii.gz -out /tmp/output_desikan_det_local_csd_native/anat/registered/t1w_csf_mask_dwi.nii.gz -init /tmp/output_desikan_det_local_csd_native/tmp/reg_m/t1wtissue2dwi_xfm.mat -interp trilinear -dof 6 -applyxfm
flirt -in /tmp/output_desikan_det_local_csd_native/anat/preproc/t1w_seg_pve_1_nores.nii.gz -ref /tmp/output_desikan_det_local_csd_native/dwi/preproc/nodif_B0.nii.gz -out /tmp/output_desikan_det_local_csd_native/anat/registered/t1w_gm_in_dwi.nii.gz -init /tmp/output_desikan_det_local_